In [1]:
import pandas as pd
import string
import numpy as np
import glob
from keras.preprocessing.sequence import pad_sequences
from nltk.stem import PorterStemmer

In [5]:
porter=PorterStemmer()
stopwords=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y']
import pickle
with open('tokenizer-200-nou.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [6]:
def clean_punct(a):        
    return a.lower().translate(str.maketrans('', '', string.punctuation))

def clean_stop(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
        if tk in stopwords:
            continue
        else:
            str1 += tk+" "
            
    return str1

def clean_stem(a):
    a_token=a.split()
    
    str1=""
    for tk in a_token:
            str1 += porter.stem(tk)+" "
            
    return str1

In [ ]:
from tensorflow import keras
model = keras.models.load_model('./Modele_LSTM/Model-200-l3-2')

count=0;

pozitive_total=0
negative_total=0
pozitive_corecte=0
negative_corecte=0
neutre_total=0
neutre_corecte=0
txtfiles = []
for file in glob.glob("Stiri_pozitive/*.txt"):
    txtfiles.append(file)
    

print("---Stiri Pozitive---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        paragrafe=list(map(clean_stem, paragrafe))
        
        while("" in paragrafe) :
            paragrafe.remove("")
        
        X_train = tokenizer.texts_to_sequences(paragrafe)
        X_train = pad_sequences(X_train, maxlen = 128)
        
        results=model.predict(X_train)
        
        sum_poz=0
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
            if np.count_nonzero(X_train[i]) > 2:
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_neg+=results[i][0]*len(str(paragrafe[i]).split())
                sum_poz+=results[i][1]*len(str(paragrafe[i]).split())
        
        print(str(file)+"  "+str(sum_neg/nr_paragrafe)+"     "+str(sum_poz/nr_paragrafe))
        
        pozitive_total+=1
        if sum_neg/nr_paragrafe < sum_poz/nr_paragrafe:
            pozitive_corecte+=1

     
print("Total stiri pozitive : "+str(pozitive_total))
print("Corecte stiri pozitive : "+str(pozitive_corecte)) 
print("Acuratete Pozitive: "+str(pozitive_corecte/pozitive_total))
print("")    

txtfiles = []
for file in glob.glob("Stiri_negative/*.txt"):
    txtfiles.append(file)
print("---Stiri Negative---")    
for file in txtfiles:
    with open(file, "r") as fis:
        stire = fis.read()
        
        paragrafe=str(stire).split('\n')
        while("" in paragrafe) :
            paragrafe.remove("")
            
        paragrafe=list(map(clean_punct, paragrafe))
        paragrafe=list(map(clean_stop, paragrafe))
        paragrafe=list(map(clean_stem, paragrafe))
        
        while("" in paragrafe) :
            paragrafe.remove("")
        
        X_train = tokenizer.texts_to_sequences(paragrafe)
        X_train = pad_sequences(X_train, maxlen = 128)
        
        results=model.predict(X_train)
        
        sum_poz=0
        sum_neg=0
        nr_paragrafe=0
        for i in range(0,len(paragrafe)):
            if np.count_nonzero(X_train[i]) > 2:
                nr_paragrafe+=1*len(str(paragrafe[i]).split())
                sum_neg+=results[i][0]*len(str(paragrafe[i]).split())
                sum_poz+=results[i][1]*len(str(paragrafe[i]).split())
                
        print(str(file)+"  "+str(sum_neg/nr_paragrafe)+"     "+str(sum_poz/nr_paragrafe))
        
        negative_total+=1
        if sum_neg/nr_paragrafe> sum_poz/nr_paragrafe:
            negative_corecte+=1
    
print("Total stiri negative : "+str(negative_total))
print("Corecte stiri negative : "+str(negative_corecte)) 
print("Acuratete negative: "+str(negative_corecte/negative_total))

print("")
print("Total stiri :" + str(pozitive_total+negative_total+neutre_total))
print("Total corecte :" + str(pozitive_corecte+negative_corecte+neutre_corecte))
print("Acuratete :" + str((pozitive_corecte+negative_corecte+neutre_corecte)/(pozitive_total+negative_total+neutre_total)))
    
    

---Stiri Pozitive---
Stiri_pozitive/34.txt  0.38680820778012276     0.6131917963425318
Stiri_pozitive/42.txt  0.5682807095233194     0.43171929820810345
Stiri_pozitive/12.txt  0.5343543034046888     0.46564570939119315
Stiri_pozitive/41.txt  0.4138100951289137     0.5861899075408776
Stiri_pozitive/45.txt  0.5205604815188122     0.4794395124291804
Stiri_pozitive/11.txt  0.5959234020289252     0.404076606429675
Stiri_pozitive/31.txt  0.4165042186498331     0.5834957878002132
Stiri_pozitive/3.txt  0.42192477621866165     0.5780752184967654
Stiri_pozitive/39.txt  0.4803257678068427     0.5196742233808874
Stiri_pozitive/17.txt  0.3046479677542662     0.6953520301060799
Stiri_pozitive/48.txt  0.41620975180141967     0.5837902405257087
Stiri_pozitive/8.txt  0.3982580633245293     0.6017419379457851
Stiri_pozitive/10.txt  0.43815935008832946     0.5618406522092481
Stiri_pozitive/16.txt  0.5231195652081321     0.4768804295958641
Stiri_pozitive/47.txt  0.4856700067156858     0.5143299916174507
S